#Commen voice for persian ASR
In this note book persian ASR will be trained on Commin voice data set.
For this perpuse I used ESPnet toolkit to train the model and observe the result.

## Check the GPU
In this section code will check if GPU is avalible or not.

In [ ]:
!nvidia-smi

Mon Aug 14 06:58:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Installing espnet on colab

In [ ]:
import torch
torch.__version__

'2.0.1+cu118'

In [ ]:
# TODO(karita): put these lines in ./espnet/tools/setup_colab.sh
# OS setup
!sudo apt-get install bc tree
!cat /etc/os-release

# espnet setup
!git clone https://github.com/espnet/espnet
!cd espnet; pip install -e .
!mkdir espnet/tools/venv/bin; touch espnet/tools/venv/bin/activate

# warp ctc setup
!git clone https://github.com/espnet/warp-ctc -b pytorch-1.13.1
!cd warp-ctc && mkdir build && cd build && cmake .. && make -j4
!cd warp-ctc/pytorch_binding && python setup.py install

# kaldi setup
!cd ./espnet/tools; git clone https://github.com/kaldi-asr/kaldi
!echo "" > ./espnet/tools/kaldi/tools/extras/check_dependencies.sh # ignore check
!chmod +x ./espnet/tools/kaldi/tools/extras/check_dependencies.sh
!cd ./espnet/tools/kaldi/tools; make sph2pipe sclite
!rm -rf espnet/tools/kaldi/tools/python
![ ! -e ubuntu16-featbin.tar.gz ] && wget https://18-198329952-gh.circle-artifacts.com/0/home/circleci/repo/ubuntu16-featbin.tar.gz
!tar -xf ./ubuntu16-featbin.tar.gz
!cp featbin/* espnet/tools/kaldi/src/featbin/

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  bc tree
0 upgraded, 2 newly installed, 0 to remove and 16 not upgraded.
Need to get 136 kB of archives.
After this operation, 336 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 bc amd64 1.07.1-3build1 [87.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 136 kB in 0s (460 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously un

In [ ]:
%cd /content/espnet/tools
!. ./activate_python.sh && python3 check_install.py | head -n 40

/content/espnet/tools
/bin/bash: line 1: ./activate_python.sh: No such file or directory


# Installing ESPnet

### Clone the ESPnet repository

In [ ]:
# It takes a few seconds
!git clone --depth 5 https://github.com/espnet/espnet

fatal: destination path 'espnet' already exists and is not an empty directory.


In [ ]:
!ls -a -l

total 20
drwxr-xr-x  1 root root 4096 Jul 26 07:55 .
drwxr-xr-x  1 root root 4096 Jul 26 07:52 ..
drwxr-xr-x  4 root root 4096 Jul 20 13:27 .config
drwxr-xr-x 17 root root 4096 Jul 26 08:02 espnet
drwxr-xr-x  1 root root 4096 Jul 20 13:28 sample_data


In [ ]:
!cd espnet && ls -a -l

total 196
drwxr-xr-x  17 root root  4096 Jul 26 08:02 .
drwxr-xr-x   1 root root  4096 Jul 26 07:55 ..
drwxr-xr-x   2 root root  4096 Jul 26 07:55 ci
-rw-r--r--   1 root root   304 Jul 26 07:55 codecov.yml
-rw-r--r--   1 root root 14960 Jul 26 07:55 CONTRIBUTING.md
-rw-r--r--   1 root root   272 Jul 26 07:55 .coveragerc
drwxr-xr-x   5 root root  4096 Jul 26 07:55 doc
drwxr-xr-x   3 root root  4096 Jul 26 07:55 docker
-rw-r--r--   1 root root   232 Jul 26 07:55 .dockerignore
drwxr-xr-x   3 root root  4096 Jul 26 07:55 .eggs
drwxr-xr-x  76 root root  4096 Jul 26 07:55 egs
drwxr-xr-x 168 root root  4096 Jul 26 07:55 egs2
drwxr-xr-x  15 root root  4096 Jul 26 07:55 espnet
drwxr-xr-x  32 root root  4096 Jul 26 07:55 espnet2
drwxr-xr-x   2 root root  4096 Jul 26 08:02 espnet.egg-info
drwxr-xr-x   8 root root  4096 Jul 26 07:55 .git
drwxr-xr-x   4 root root  4096 Jul 26 07:55 .github
-rw-r--r--   1 root root  1292 Jul 26 07:55 .gitignore
-rw-r--r--   1 root root     0 Jul 26 07:55 .gitmodules

### Create an enviromet with python 3.9

In [ ]:
%cd /content/espnet/tools
!./setup_anaconda.sh anaconda espnet 3.9

/content/espnet/tools
--2023-07-26 08:04:31--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 103219356 (98M) [application/x-sh]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>]  98.44M   215MB/s    in 0.5s    

2023-07-26 08:04:32 (215 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [103219356/103219356]

PREFIX=/content/espnet/tools/anaconda
Unpacking payload ...

Installing base environment...





Preparing transaction: - \ | / - done
Executing transaction: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    

### Install espnet and dependencies

In [ ]:
# It may take 12 minutes
%cd /content/espnet/tools
!make TH_VERSION=1.13.1

Streaming output truncated to the last 5000 lines.







pytorch-1.13.1       | 1.14 GB   | :  54% 0.5444165895217065/1 [00:32<00:13, 28.55s/it]














pytorch-1.13.1       | 1.14 GB   | :  55% 0.5479968713727031/1 [00:33<00:13, 28.83s/it]














pytorch-1.13.1       | 1.14 GB   | :  55% 0.5519779310428413/1 [00:33<00:12, 28.14s/it]














pytorch-1.13.1       | 1.14 GB   | :  56% 0.5559456314523413/1 [00:33<00:12, 27.41s/it]














pytorch-1.13.1       | 1.14 GB   | :  56% 0.5620240950426529/1 [00:33<00:10, 23.06s/it]














pytorch-1.13.1       | 1.14 GB   | :  57% 0.567968966026584/1 [00:33<00:09, 21.26s/it] 














pytorch-1.13.1       | 1.14 GB   | :  57% 0.5727115035530909/1 [00:33<00:09, 22.31s/it]














pytorch-1.13.1       | 1.14 GB   | :  58% 0.5788968412285067/1 [00:33<00:08, 20.17s/it]














pytorch-1.13.1       | 1.14 GB   | :  59% 0.5852558492922173/1 [00:33<00:07, 18.66s/it]














pytorch-1.13.1       | 1

In [ ]:
# s3prl and fairseq are necessary if you want to use self-supervised pre-trained models
# It takes 50s
%cd /content/espnet/tools

!. ./activate_python.sh && ./installers/install_s3prl.sh      # install s3prl to use SSLRs
!. ./activate_python.sh && ./installers/install_fairseq.sh    # install s3prl to use Wav2Vec2 / HuBERT model series

/content/espnet/tools
cuda_version=117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 47.6 MB/s eta 0:00:00
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/21/02/ae8e595f45b6c8edee07913892b3b41f5f5f273962ad98851dc6a564bbb9/transformers-4.31.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 76.8 MB/s eta 0:00:00
  Created wheel for s3prl: filename=s3prl-0.4.10-py3-none-any.whl size=864129 sha256=0487fd146a334bbce0cd3a3926a4b03cc755f2970f0fc1e1c132a0e4acf1cf23
  Stored in directory: /root/.cache/pip/wheels/a4/a8/bb/9664facfc6106635e28531aa2298

In [ ]:
%cd /content/espnet/tools
!. ./activate_python.sh && python3 check_install.py | head -n 40

/content/espnet/tools
[x] python=3.9.17 (main, Jul  5 2023, 20:41:20)  [GCC 11.2.0]

Python modules:
[x] torch=1.13.1
[x] torch cuda=11.7
[x] torch cudnn=8500
[x] torch nccl
[x] chainer=6.0.0
[ ] chainer cuda
[ ] chainer cudnn
[ ] cupy
[x] torchaudio=0.13.1
[x] torch_optimizer=0.3.0
[ ] warprnnt_pytorch
[ ] chainer_ctc
[ ] pyopenjtalk
[ ] tdmelodic_pyopenjtalk
[ ] kenlm
[ ] mmseg
[x] espnet=202304
[x] numpy=1.23.5
[x] fairseq
[ ] phonemizer
[ ] gtn
[x] s3prl=0.4.10
[x] transformers=4.31.0
[ ] speechbrain
[ ] k2
[ ] longformer
[ ] nlg-eval
[ ] datasets
[ ] pykeops
[ ] whisper
[ ] RawNet3
[ ] reazonspeech
[ ] muskits

Executables:
[ ] sclite
[ ] sph2pipe


# Commen Voice recipe implementation

### Observe common voice directory in ESPnet

In [ ]:
%cd /content/espnet/egs2/commonvoice/asr1/
!ls

/content/espnet/egs2/commonvoice/asr1
asr.sh	db.sh				    path.sh    run.sh	utils
cmd.sh	Fusing_ASR_frontends_in_Espnet2.md  pyscripts  scripts
conf	local				    README.md  steps


In [ ]:
! ./run.sh --help

2023-08-14T07:03:18 (asr.sh:277:main) ./asr.sh --ngpu 4 --lang cy --local_data_opts --lang cy --use_lm true --lm_config conf/train_lm.yaml --token_type bpe --nbpe 150 --feats_type raw --speed_perturb_factors 0.9 1.0 1.1 --asr_config conf/tuning/train_asr_conformer5.yaml --inference_config conf/decode_asr.yaml --train_set train_cy --valid_set dev_cy --test_sets dev_cy test_cy --bpe_train_text data/train_cy/text --lm_train_text data/train_cy/text --help
Usage: ./asr.sh --train-set "<train_set_name>" --valid-set "<valid_set_name>" --test_sets "<test_set_names>"

Options:
    # General configuration
    --stage          # Processes starts from the specified stage (default="1").
    --stop_stage     # Processes is stopped at the specified stage (default="10000").
    --skip_stages    # Spicify the stage to be skipped (default="").
    --skip_data_prep # Skip data preparation stages (default="false").
    --skip_train     # Skip training stages (default="false").
    --skip_eval      # Skip 

In [ ]:
!cat run.sh

#!/usr/bin/env bash
# Set bash to 'debug' mode, it will exit on :
# -e 'error', -u 'undefined variable', -o ... 'error in pipeline', -x 'print commands',
set -e
set -u
set -o pipefail

lang=cy # en de fr cy tt kab ca zh-TW it fa eu es ru tr nl eo zh-CN rw pt zh-HK cs pl uk

train_set=train_"$(echo "${lang}" | tr - _)"
train_dev=dev_"$(echo "${lang}" | tr - _)"
test_set="${train_dev} test_$(echo ${lang} | tr - _)"

asr_config=conf/tuning/train_asr_conformer5.yaml
lm_config=conf/train_lm.yaml
inference_config=conf/decode_asr.yaml

if [[ "zh" == *"${lang}"* ]]; then
  nbpe=2500
elif [[ "fr" == *"${lang}"* ]]; then
  nbpe=350
elif [[ "es" == *"${lang}"* ]]; then
  nbpe=235
else
  nbpe=150
fi

./asr.sh \
    --ngpu 4 \
    --lang "${lang}" \
    --local_data_opts "--lang ${lang}" \
    --use_lm true \
    --lm_config "${lm_config}" \
    --token_type bpe \
    --nbpe $nbpe \
    --feats_type raw \
    --speed_perturb_factors "0.9 1.0 1.1" \
    --asr_config "${asr_config}" \
    --inference

# Load the data

**Important Note:** there is problem in changing the language here. the language should be hard coded change change in files. ./local/data.sh should be changed

In [ ]:
!rm -rf /content/espnet/egs2/commonvoice/asr1/downloads
!rm -rf /content/espnet/egs2/commonvoice/asr1/data
!rm -rf /content/espnet/egs2/commonvoice/asr1/dump
#!mkdir downloads
! pwd && du -h --max-depth=1

/content/espnet/egs2/commonvoice/asr1
56K	./conf
12K	./local
104K	.


In [ ]:
! ./command.sh

/bin/bash: line 1: ./command.sh: No such file or directory


In [ ]:
# a few seconds
!./asr.sh --stop_stage 2 --train_set train_fa --valid_set dev_fa --test_sets test_fa --lang "fa" --local_data_opts "--lang fa"

2023-07-30T08:15:00 (asr.sh:277:main) ./asr.sh --stop_stage 2 --train_set train_fa --valid_set dev_fa --test_sets test_fa --lang fa --local_data_opts --lang fa
[INFO] /content/espnet/egs2/commonvoice/asr1/../../../tools/activate_python.sh is not present
2023-07-30T08:15:00 (asr.sh:559:main) Skipped stages:  9 14 15 16 
2023-07-30T08:15:00 (asr.sh:566:main) Stage 1: Data preparation for data/train_fa, data/dev_fa, etc.
[INFO] /content/espnet/egs2/commonvoice/asr1/../../../tools/activate_python.sh is not present
2023-07-30T08:15:00 (data.sh:43:main) data preparation started
2023-07-30T08:15:00 (data.sh:46:main) stage1: Download data to downloads
2023-07-30T08:15:00 (data.sh:48:main) The default data of this recipe is from commonvoice 5.1, for newer version, you need to register at          https://commonvoice.mozilla.org/
local/download_and_untar.sh: data was already successfully extracted, nothing to do.
2023-07-30T08:15:00 (data.sh:53:main) stage2: Preparing data for commonvoice
utils/

# Pre-prosecessing

In [ ]:
!rm -r ./data/
!rm -r ./dump/

rm: cannot remove './dump/': No such file or directory


In [ ]:
!pwd
!du /content/espnet/egs2/commonvoice/asr1/data -h

/content/espnet/egs2/commonvoice/asr1
3.7M	/content/espnet/egs2/commonvoice/asr1/data/test_fa/.backup
8.0M	/content/espnet/egs2/commonvoice/asr1/data/test_fa
214M	/content/espnet/egs2/commonvoice/asr1/data/train_fa/.backup
485M	/content/espnet/egs2/commonvoice/asr1/data/train_fa
216M	/content/espnet/egs2/commonvoice/asr1/data/validated_fa/.backup
468M	/content/espnet/egs2/commonvoice/asr1/data/validated_fa
3.7M	/content/espnet/egs2/commonvoice/asr1/data/dev_fa/.backup
8.0M	/content/espnet/egs2/commonvoice/asr1/data/dev_fa
969M	/content/espnet/egs2/commonvoice/asr1/data


In [ ]:
!utils/subset_data_dir.sh "./data/train_fa/" 10000 "./data/train_fa2/"
!utils/subset_data_dir.sh "./data/validated_fa/" 10000 "./data/validated_fa2/"
!utils/subset_data_dir.sh "./data/dev_fa/" 1000 "./data/dev_fa2/"
!utils/subset_data_dir.sh "./data/test_fa/" 1000 "./data/test_fa2/"

utils/subset_data_dir.sh: reducing #utt from 236758 to 10000
utils/subset_data_dir.sh: reducing #utt from 244736 to 10000
utils/subset_data_dir.sh: reducing #utt from 3989 to 1000
utils/subset_data_dir.sh: reducing #utt from 3989 to 1000


In [ ]:
! rm -r /content/espnet/egs2/commonvoice/asr1/data/dev_fa
! rm -r /content/espnet/egs2/commonvoice/asr1/data/test_fa
! rm -r /content/espnet/egs2/commonvoice/asr1/data/train_fa
! rm -r /content/espnet/egs2/commonvoice/asr1/data/validated_fa

In [ ]:
!mv /content/espnet/egs2/commonvoice/asr1/data/dev_fa2 /content/espnet/egs2/commonvoice/asr1/data/dev_fa
!mv /content/espnet/egs2/commonvoice/asr1/data/test_fa2 /content/espnet/egs2/commonvoice/asr1/data/test_fa
!mv /content/espnet/egs2/commonvoice/asr1/data/train_fa2 /content/espnet/egs2/commonvoice/asr1/data/train_fa
!mv /content/espnet/egs2/commonvoice/asr1/data/validated_fa2 /content/espnet/egs2/commonvoice/asr1/data/validated_fa

In [ ]:
!du /content/espnet/egs2/commonvoice/asr1/data -h

1.1M	/content/espnet/egs2/commonvoice/asr1/data/test_fa
9.0M	/content/espnet/egs2/commonvoice/asr1/data/train_fa
9.0M	/content/espnet/egs2/commonvoice/asr1/data/validated_fa
1020K	/content/espnet/egs2/commonvoice/asr1/data/dev_fa
20M	/content/espnet/egs2/commonvoice/asr1/data


Change format to wav

In [ ]:
!./asr.sh --stage 3 --stop_stage 3 --train_set train_fa --valid_set dev_fa --test_sets test_fa --lang "fa" --local_data_opts "--lang fa" --audio_format wav

2023-07-30T08:17:09 (asr.sh:277:main) ./asr.sh --stage 3 --stop_stage 3 --train_set train_fa --valid_set dev_fa --test_sets test_fa --lang fa --local_data_opts --lang fa --audio_format wav
[INFO] /content/espnet/egs2/commonvoice/asr1/../../../tools/activate_python.sh is not present
2023-07-30T08:17:10 (asr.sh:559:main) Skipped stages:  9 14 15 16 
2023-07-30T08:17:10 (asr.sh:609:main) Stage 3: Format wav.scp: data/ -> dump/raw
utils/copy_data_dir.sh: copied data from data/train_fa to dump/raw/org/train_fa
utils/validate_data_dir.sh: Successfully validated data-directory dump/raw/org/train_fa
2023-07-30T08:17:10 (format_wav_scp.sh:45:main) scripts/audio/format_wav_scp.sh --nj 32 --cmd run.pl --audio-format wav --fs 16k --multi-columns-input false --multi-columns-output false data/train_fa/wav.scp dump/raw/org/train_fa
[INFO] /content/espnet/egs2/commonvoice/asr1/../../../tools/activate_python.sh is not present
2023-07-30T08:17:10 (format_wav_scp.sh:117:main) [info]: without segments
202

Delete long short data

In [ ]:
!./asr.sh --stage 4 --stop_stage 4 --train_set train_fa --valid_set dev_fa --test_sets test_fa --lang "fa" --local_data_opts "--lang fa"

2023-07-30T08:43:43 (asr.sh:277:main) ./asr.sh --stage 4 --stop_stage 4 --train_set train_fa --valid_set dev_fa --test_sets test_fa --lang fa --local_data_opts --lang fa
[INFO] /content/espnet/egs2/commonvoice/asr1/../../../tools/activate_python.sh is not present
2023-07-30T08:43:43 (asr.sh:559:main) Skipped stages:  9 14 15 16 
2023-07-30T08:43:43 (asr.sh:794:main) Stage 4: Remove long/short data: dump/raw/org -> dump/raw
utils/copy_data_dir.sh: copied data from dump/raw/org/train_fa to dump/raw/train_fa
utils/validate_data_dir.sh: Successfully validated data-directory dump/raw/train_fa
fix_data_dir.sh: kept all 10000 utterances.
fix_data_dir.sh: old files are kept in dump/raw/train_fa/.backup
utils/copy_data_dir.sh: copied data from dump/raw/org/dev_fa to dump/raw/dev_fa
utils/validate_data_dir.sh: Successfully validated data-directory dump/raw/dev_fa
fix_data_dir.sh: kept all 1000 utterances.
fix_data_dir.sh: old files are kept in dump/raw/dev_fa/.backup
2023-07-30T08:43:45 (asr.sh:

Tokenization

In [ ]:
!./asr.sh --stage 5 --stop_stage 5 --train_set train_fa --valid_set dev_fa --test_sets test_fa --lang "fa" --local_data_opts "--lang fa" --nbpe 80

2023-07-30T08:44:08 (asr.sh:277:main) ./asr.sh --stage 5 --stop_stage 5 --train_set train_fa --valid_set dev_fa --test_sets test_fa --lang fa --local_data_opts --lang fa --nbpe 80
[INFO] /content/espnet/egs2/commonvoice/asr1/../../../tools/activate_python.sh is not present
2023-07-30T08:44:08 (asr.sh:559:main) Skipped stages:  9 14 15 16 
2023-07-30T08:44:08 (asr.sh:872:main) Stage 5: Generate token_list from dump/raw/org/train_fa/text using BPE
sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=data/fa_token_list/bpe_unigram80/train.txt --vocab_size=80 --model_type=unigram --model_prefix=data/fa_token_list/bpe_unigram80/bpe --character_coverage=1.0 --input_sentence_size=100000000
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: data/fa_token_list/bpe_unigram80/train.txt
  input_format: 
  model_prefix: data/fa_token_list/bpe_unigram80/bpe
  model_type: UNIGRAM
  vocab_size: 80
  self_test_sample_size: 0
  character_coverage: 1
  inpu

# Configs


espnet/egs2/librispeech/asr1/conf/tuning
/train_asr_e_branchformer.yaml
```
# Trained with A100 (40 GB) x 8 GPUs. It takes 40 minutes per epoch.
encoder: e_branchformer
encoder_conf:
    output_size: 512
    attention_heads: 8
    attention_layer_type: rel_selfattn
    pos_enc_layer_type: rel_pos
    rel_pos_type: latest
    cgmlp_linear_units: 3072
    cgmlp_conv_kernel: 31
    use_linear_after_conv: false
    gate_activation: identity
    num_blocks: 17
    dropout_rate: 0.1
    positional_dropout_rate: 0.1
    attention_dropout_rate: 0.1
    input_layer: conv2d
    layer_drop_rate: 0.1
    linear_units: 1024
    positionwise_layer_type: linear
    macaron_ffn: true
    use_ffn: true
    merge_conv_kernel: 31

decoder: transformer
decoder_conf:
    attention_heads: 8
    linear_units: 2048
    num_blocks: 6
    dropout_rate: 0.1
    positional_dropout_rate: 0.1
    self_attention_dropout_rate: 0.1
    src_attention_dropout_rate: 0.1
    layer_drop_rate: 0.2

model_conf:
    ctc_weight: 0.3
    lsm_weight: 0.1
    length_normalized_loss: false

frontend_conf:
    n_fft: 512
    hop_length: 160

use_amp: true
unused_parameters: true
num_workers: 8
batch_type: numel
batch_bins: 140000000
accum_grad: 1
max_epoch: 80
patience: none
init: none
best_model_criterion:
-   - valid
    - acc
    - max
keep_nbest_models: 10
nbest_averaging_interval: 10

optim: adam
optim_conf:
    lr: 0.002
    weight_decay: 0.000001
scheduler: warmuplr
scheduler_conf:
    warmup_steps: 40000

specaug: specaug
specaug_conf:
    apply_time_warp: true
    time_warp_window: 5
    time_warp_mode: bicubic
    apply_freq_mask: true
    freq_mask_width_range:
    - 0
    - 27
    num_freq_mask: 2
    apply_time_mask: true
    time_mask_width_ratio_range:
    - 0.
    - 0.05
    num_time_mask: 10
```
espnet/egs2/librispeech/asr1/conf/tuning
/train_asr_branchformer_hop_length160_e18_linear3072.yaml
```
# Trained with A6000 (48 GB) x 4 GPUs. It takes 100 minutes per epoch.
encoder: branchformer
encoder_conf:
    output_size: 512
    use_attn: true
    attention_heads: 8
    attention_layer_type: rel_selfattn
    pos_enc_layer_type: rel_pos
    rel_pos_type: latest
    use_cgmlp: true
    cgmlp_linear_units: 3072
    cgmlp_conv_kernel: 31
    use_linear_after_conv: false
    gate_activation: identity
    merge_method: concat
    cgmlp_weight: 0.5               # used only if merge_method is "fixed_ave"
    attn_branch_drop_rate: 0.0      # used only if merge_method is "learned_ave"
    num_blocks: 18
    dropout_rate: 0.1
    positional_dropout_rate: 0.1
    attention_dropout_rate: 0.1
    input_layer: conv2d
    stochastic_depth_rate: 0.0

decoder: transformer
decoder_conf:
    attention_heads: 8
    linear_units: 2048
    num_blocks: 6
    dropout_rate: 0.1
    positional_dropout_rate: 0.1
    self_attention_dropout_rate: 0.1
    src_attention_dropout_rate: 0.1

model_conf:
    ctc_weight: 0.3
    lsm_weight: 0.1
    length_normalized_loss: false

frontend_conf:
    n_fft: 512
    hop_length: 160

use_amp: true
num_workers: 4
batch_type: numel
batch_bins: 35000000
accum_grad: 4
max_epoch: 70
patience: none
init: none
best_model_criterion:
-   - valid
    - acc
    - max
keep_nbest_models: 10
nbest_averaging_interval: 10

optim: adam
optim_conf:
    lr: 0.0025
    weight_decay: 0.000001
scheduler: warmuplr
scheduler_conf:
    warmup_steps: 40000

specaug: specaug
specaug_conf:
    apply_time_warp: true
    time_warp_window: 5
    time_warp_mode: bicubic
    apply_freq_mask: true
    freq_mask_width_range:
    - 0
    - 27
    num_freq_mask: 2
    apply_time_mask: true
    time_mask_width_ratio_range:
    - 0.
    - 0.05
    num_time_mask: 10
```
Toutorial Branchformer
```
batch_type: folded
batch_size: 64
accum_grad: 1    # gradient accumulation steps
max_epoch: 100
patience: none
init: xavier_uniform
best_model_criterion:  # criterion to save best models
-   - valid
    - acc
    - max
keep_nbest_models: 10  # save nbest models and average these checkpoints
use_amp: true    # whether to use automatic mixed precision
num_att_plot: 0  # do not save attention plots to save time in the demo
num_workers: 2   # number of workers in dataloader

encoder: branchformer
encoder_conf:
    output_size: 256
    use_attn: true
    attention_heads: 4
    attention_layer_type: rel_selfattn
    pos_enc_layer_type: rel_pos
    rel_pos_type: latest
    use_cgmlp: true
    cgmlp_linear_units: 1024
    cgmlp_conv_kernel: 31
    use_linear_after_conv: false
    gate_activation: identity
    merge_method: concat
    cgmlp_weight: 0.5               # used only if merge_method is "fixed_ave"
    attn_branch_drop_rate: 0.0      # used only if merge_method is "learned_ave"
    num_blocks: 12
    dropout_rate: 0.1
    positional_dropout_rate: 0.1
    attention_dropout_rate: 0.1
    input_layer: conv2d
    stochastic_depth_rate: 0.0

decoder: transformer
decoder_conf:
    attention_heads: 4
    linear_units: 1024
    num_blocks: 3
    dropout_rate: 0.1
    positional_dropout_rate: 0.1
    self_attention_dropout_rate: 0.1
    src_attention_dropout_rate: 0.1

model_conf:
    ctc_weight: 0.3  # joint CTC/attention training
    lsm_weight: 0.1  # label smoothing weight
    length_normalized_loss: false

optim: adam
optim_conf:
    lr: 0.001
scheduler: warmuplr  # linearly increase and exponentially decrease
scheduler_conf:
    warmup_steps: 800
```



# Train laguage model

In [ ]:
import espnet

## Download dataset

In [ ]:
%cd /content/
! wget https://storage.googleapis.com/danielk-files/farsi-text/merged_files/fawiki_merged.txt

/content
--2023-08-14 07:03:19--  https://storage.googleapis.com/danielk-files/farsi-text/merged_files/fawiki_merged.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.11.128, 108.177.12.128, 108.177.13.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.11.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 787234518 (751M) [text/plain]
Saving to: ‘fawiki_merged.txt’

fawiki_merged.txt   100%[===================>] 750.76M   150MB/s    in 5.2s    

2023-08-14 07:03:24 (145 MB/s) - ‘fawiki_merged.txt’ saved [787234518/787234518]



Observe the data

In [ ]:
!cd /content/data && ls -l && cat fawiki_merged.txt

/bin/bash: line 1: cd: /content/data: No such file or directory


In [ ]:
%cd /content/
!mkdir data

/content


In [ ]:
%cd /content/espnet/espnet2/bin

/content/espnet/espnet2/bin


In [ ]:
!ls -l

total 12
drwxr-xr-x  2 root root 4096 Aug 14 07:09 data
drwxr-xr-x 17 root root 4096 Aug 14 07:01 espnet
drwxr-xr-x  1 root root 4096 Aug 10 19:19 sample_data


## Tokenaization

In [ ]:
!python3 -m tokenize_text --output "/content/token.txt" --input "/content/data/fawiki_merged.txt" --token_type "char" --vocabulary_size "150" --write_vocabulary "true" --field "2-" --add_symbol "<blank>:0" --add_symbol "<unk>:1" --add_symbol "<sos/eos>:-1"

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
/usr/bin/python3 /content/espnet/espnet2/bin/tokenize_text.py --output /content/token.txt --input /content/data/fawiki_merged.txt --token_type char --vocabulary_size 150 --write_vocabulary true --field 2- --add_symbol '<blank>:0' --add_symbol '<unk>:1' --add_symbol '<sos/eos>:-1'
2023-08-14 07:32:31,955 (tokenize_text:173) INFO: OOV rate = 0.05656557774810202 %


## Data prepration

خشک خشک

In [ ]:
!python3 lm_train.py --output_dir "/content/" --token_list "/content/token.txt"  --config "/content/espnet/egs2/commonvoice/asr1/conf/train_lm.yaml" --token_type "char" --train_data_path_and_name_and_type "/content/data/test.text,text,text" --valid_data_path_and_name_and_type "/content/data/valid.text,text,text" --collect_stats true --use_preprocessor true --batch_type unsorted

/usr/bin/python3 lm_train.py --output_dir /content/ --token_list /content/token.txt --config /content/espnet/egs2/commonvoice/asr1/conf/train_lm.yaml --token_type char --train_data_path_and_name_and_type /content/data/test.text,text,text --valid_data_path_and_name_and_type /content/data/valid.text,text,text --collect_stats true --use_preprocessor true --batch_type unsorted
[2a8c802bd245] 2023-08-14 08:16:57,855 (lm:191) INFO: Vocabulary size: 150
[2a8c802bd245] 2023-08-14 08:16:58,009 (abs_task:1229) INFO: pytorch.version=2.0.1+cu118, cuda.available=True, cudnn.version=8700, cudnn.benchmark=False, cudnn.deterministic=True
[2a8c802bd245] 2023-08-14 08:16:58,009 (abs_task:1230) INFO: Model structure:
ESPnetLanguageModel(
  (lm): SequentialRNNLM(
    (drop): Dropout(p=0.0, inplace=False)
    (encoder): Embedding(150, 650, padding_idx=0)
    (rnn): LSTM(650, 650, num_layers=2, batch_first=True)
    (decoder): Linear(in_features=650, out_features=150, bias=True)
  )
)

Model summary:
    Cl

with commenvoice

In [ ]:
%cd /content/espnet/egs2/commonvoice/asr1
!cp /content/data/test.text dump/raw/lm_train.txt
!cp /content/data/valid.text dump/raw/org/dev_fa/text
!cp /content/token.txt data/fa_token_list/char/tokens.txt

/content/espnet/egs2/commonvoice/asr1


In [ ]:
!./asr.sh --stage 6 --stop-stage 6 --ngpu 1 --lang "fa" --local_data_opts "--lang fa" --use_lm true --lm_config "conf/train_lm.yaml"  --token_type char  --nbpe 150 --feats_type raw  --speed_perturb_factors "0.9 1.0 1.1"  --asr_config "conf/train_lm.yaml"  --inference_config "conf/decode_asr.yaml" --train_set "train_fa"  --valid_set "dev_fa"  --test_sets "test_fa"

2023-08-14T08:36:33 (asr.sh:277:main) ./asr.sh --stage 6 --stop-stage 6 --ngpu 1 --lang fa --local_data_opts --lang fa --use_lm true --lm_config conf/train_lm.yaml --token_type char --nbpe 150 --feats_type raw --speed_perturb_factors 0.9 1.0 1.1 --asr_config conf/train_lm.yaml --inference_config conf/decode_asr.yaml --train_set train_fa --valid_set dev_fa --test_sets test_fa
[INFO] /content/espnet/egs2/commonvoice/asr1/../../../tools/activate_python.sh is not present
2023-08-14T08:36:33 (asr.sh:559:main) Skipped stages:  9 14 15 16 
2023-08-14T08:36:33 (asr.sh:1006:main) Stage 6: LM collect stats: train_set=dump/raw/lm_train.txt, dev_set=dump/raw/org/dev_fa/text
2023-08-14T08:36:33 (asr.sh:1038:main) Generate 'exp/lm_stats_fa_char/run.sh'. You can resume the process from stage 6 using this script
2023-08-14T08:36:33 (asr.sh:1042:main) LM collect-stats started... log: 'exp/lm_stats_fa_char/logdir/stats.*.log'
/usr/bin/python3 /content/espnet/espnet2/bin/aggregate_stats_dirs.py --input_d

## Train laguage model

In [ ]:
!./asr.sh --stage 7 --stop-stage 7 --ngpu 1 --lang "fa" --local_data_opts "--lang fa" --use_lm true --lm_config "conf/train_lm.yaml"  --token_type char  --nbpe 150 --feats_type raw  --speed_perturb_factors "0.9 1.0 1.1"  --asr_config "conf/train_lm.yaml"  --inference_config "conf/decode_asr.yaml" --train_set "train_fa"  --valid_set "dev_fa"  --test_sets "test_fa"

2023-08-14T08:37:45 (asr.sh:277:main) ./asr.sh --stage 7 --stop-stage 7 --ngpu 1 --lang fa --local_data_opts --lang fa --use_lm true --lm_config conf/train_lm.yaml --token_type char --nbpe 150 --feats_type raw --speed_perturb_factors 0.9 1.0 1.1 --asr_config conf/train_lm.yaml --inference_config conf/decode_asr.yaml --train_set train_fa --valid_set dev_fa --test_sets test_fa
[INFO] /content/espnet/egs2/commonvoice/asr1/../../../tools/activate_python.sh is not present
2023-08-14T08:37:45 (asr.sh:559:main) Skipped stages:  9 14 15 16 
2023-08-14T08:37:45 (asr.sh:1083:main) Stage 7: LM Training: train_set=dump/raw/lm_train.txt, dev_set=dump/raw/org/dev_fa/text
2023-08-14T08:37:45 (asr.sh:1120:main) Generate 'exp/lm_train_lm_fa_char/run.sh'. You can resume the process from stage 7 using this script
2023-08-14T08:37:45 (asr.sh:1123:main) LM training started... log: 'exp/lm_train_lm_fa_char/train.log'
2023-08-14 08:37:45,824 (launch:94) INFO: /usr/bin/python3 /content/espnet/espnet2/bin/laun

In [ ]:
!python3 lm_train.py --output_dir "/content/" --token_list "/content/token.txt"  --config "/content/espnet/egs2/commonvoice/asr1/conf/train_lm.yaml" --token_type "char" --train_data_path_and_name_and_type "/content/data/test.text,text,text" --valid_data_path_and_name_and_type "/content/data/valid.text,text,text" --collect_stats true --use_preprocessor true --batch_type unsorted

In [ ]:
!./asr.sh --stage 7 --stop-stage 7 --ngpu 1 --lang "fa" --local_data_opts "--lang fa" --use_lm true --lm_config "conf/train_lm.yaml"  --token_type bpe  --nbpe 154 --feats_type raw  --speed_perturb_factors "0.9 1.0 1.1"  --asr_config "conf/train_lm.yaml"  --inference_config "conf/train_lm.yaml"  --train_set "/content/data"  --valid_set "/content/data"  --test_sets "/content/data"  --bpe_train_text "/content/data/parsa.txt"  --lm_train_text "/content/data/parsa.txt" --skip-train true

2023-08-13T11:56:22 (asr.sh:277:main) ./asr.sh --stage 7 --stop-stage 7 --ngpu 1 --lang fa --local_data_opts --lang fa --use_lm true --lm_config conf/train_lm.yaml --token_type bpe --nbpe 154 --feats_type raw --speed_perturb_factors 0.9 1.0 1.1 --asr_config conf/train_lm.yaml --inference_config conf/train_lm.yaml --train_set /content/data --valid_set /content/data --test_sets /content/data --bpe_train_text /content/data/parsa.txt --lm_train_text /content/data/parsa.txt --skip-train true
[INFO] /content/espnet/egs2/commonvoice/asr1/../../../tools/activate_python.sh is not present
2023-08-13T11:56:22 (asr.sh:304:main) Error: train_set and valid_set must be different. --train_set /content/data --valid_set /content/data


In [ ]:
!./asr.sh --stage 1 --stop-stage 1 --ngpu 1 --lang "fa" --local_data_opts "--lang fa" --use_lm true --lm_config "conf/train_lm.yaml"  --token_type bpe  --nbpe 154 --feats_type raw  --speed_perturb_factors "0.9 1.0 1.1"  --asr_config "conf/train_lm.yaml"  --inference_config "conf/train_lm.yaml"  --train_set "train_fa"  --valid_set "dev_fa"  --test_sets "test_fa"  --bpe_train_text "/content/data/parsa.txt"  --lm_train_text "/content/data/parsa.txt"

2023-08-13T12:03:14 (asr.sh:277:main) ./asr.sh --stage 1 --stop-stage 1 --ngpu 1 --lang fa --local_data_opts --lang fa --use_lm true --lm_config conf/train_lm.yaml --token_type bpe --nbpe 154 --feats_type raw --speed_perturb_factors 0.9 1.0 1.1 --asr_config conf/train_lm.yaml --inference_config conf/train_lm.yaml --train_set train_fa --valid_set dev_fa --test_sets test_fa --bpe_train_text /content/data/parsa.txt --lm_train_text /content/data/parsa.txt
[INFO] /content/espnet/egs2/commonvoice/asr1/../../../tools/activate_python.sh is not present
2023-08-13T12:03:14 (asr.sh:559:main) Skipped stages:  9 14 15 16 
2023-08-13T12:03:14 (asr.sh:566:main) Stage 1: Data preparation for data/train_fa, data/dev_fa, etc.
[INFO] /content/espnet/egs2/commonvoice/asr1/../../../tools/activate_python.sh is not present
2023-08-13T12:03:14 (data.sh:43:main) data preparation started
2023-08-13T12:03:14 (data.sh:46:main) stage1: Download data to downloads
2023-08-13T12:03:14 (data.sh:48:main) The default da

In [ ]:
!./asr.sh --stage 5 --stop-stage 5 --ngpu 1 --lang "fa" --local_data_opts "--lang fa" --use_lm true --lm_config "conf/train_lm.yaml"  --token_type bpe  --nbpe 22 --feats_type raw  --speed_perturb_factors "0.9 1.0 1.1"  --asr_config "conf/train_lm.yaml"  --inference_config "conf/train_lm.yaml"  --train_set "train_fa"  --valid_set "dev_fa"  --test_sets "test_fa"  --bpe_train_text "/content/data/parsa.txt"  --lm_train_text "/content/data/parsa.txt"

2023-08-13T12:18:38 (asr.sh:277:main) ./asr.sh --stage 5 --stop-stage 5 --ngpu 1 --lang fa --local_data_opts --lang fa --use_lm true --lm_config conf/train_lm.yaml --token_type bpe --nbpe 22 --feats_type raw --speed_perturb_factors 0.9 1.0 1.1 --asr_config conf/train_lm.yaml --inference_config conf/train_lm.yaml --train_set train_fa --valid_set dev_fa --test_sets test_fa --bpe_train_text /content/data/parsa.txt --lm_train_text /content/data/parsa.txt
[INFO] /content/espnet/egs2/commonvoice/asr1/../../../tools/activate_python.sh is not present
2023-08-13T12:18:38 (asr.sh:559:main) Skipped stages:  9 14 15 16 
2023-08-13T12:18:38 (asr.sh:872:main) Stage 5: Generate token_list from /content/data/parsa.txt using BPE
sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=data/fa_token_list/bpe_unigram22/train.txt --vocab_size=22 --model_type=unigram --model_prefix=data/fa_token_list/bpe_unigram22/bpe --character_coverage=1.0 --input_sentence_size=100000000
sentencepiece_trainer.cc

In [ ]:
!./asr.sh --stage 3 --stop-stage 3 --ngpu 1 --lang "fa" --local_data_opts "--lang fa" --use_lm true --lm_config "conf/train_lm.yaml"  --token_type bpe  --nbpe 22 --feats_type raw  --speed_perturb_factors "0.9 1.0 1.1"  --asr_config "conf/train_lm.yaml"  --inference_config "conf/train_lm.yaml"  --train_set "train_fa"  --valid_set "dev_fa"  --test_sets "test_fa"  --bpe_train_text "/content/data/parsa.txt"  --lm_train_text "/content/data/parsa.txt" --skip_train true

2023-08-13T12:32:43 (asr.sh:277:main) ./asr.sh --stage 3 --stop-stage 3 --ngpu 1 --lang fa --local_data_opts --lang fa --use_lm true --lm_config conf/train_lm.yaml --token_type bpe --nbpe 22 --feats_type raw --speed_perturb_factors 0.9 1.0 1.1 --asr_config conf/train_lm.yaml --inference_config conf/train_lm.yaml --train_set train_fa --valid_set dev_fa --test_sets test_fa --bpe_train_text /content/data/parsa.txt --lm_train_text /content/data/parsa.txt --skip_train true
[INFO] /content/espnet/egs2/commonvoice/asr1/../../../tools/activate_python.sh is not present
2023-08-13T12:32:43 (asr.sh:559:main) Skipped stages:  2 4 5 6 7 8 9 10 11 14 15 16 
2023-08-13T12:32:43 (asr.sh:609:main) Stage 3: Format wav.scp: data/ -> dump/raw
utils/copy_data_dir.sh: copied data from data/test_fa to dump/raw/test_fa
utils/validate_data_dir.sh: Successfully validated data-directory dump/raw/test_fa
2023-08-13T12:32:44 (format_wav_scp.sh:45:main) scripts/audio/format_wav_scp.sh --nj 32 --cmd run.pl --audio-f

In [ ]:
!cp /content/data/fawiki_merged.txt dump/raw/org/dev_fa/text

In [ ]:
!mkdir dump/raw/org/dev_fa

In [ ]:
!./asr.sh --stage 5 --stop-stage 5 --ngpu 1 --lang "fa" --local_data_opts "--lang fa" --use_lm true --lm_config "conf/train_lm.yaml"  --token_type char  --nbpe 150 --feats_type raw  --speed_perturb_factors "0.9 1.0 1.1"  --asr_config "conf/train_lm.yaml"  --inference_config "conf/train_lm.yaml" --train_set "train_fa"  --valid_set "dev_fa"  --test_sets "test_fa"

2023-08-13T12:45:02 (asr.sh:277:main) ./asr.sh --stage 5 --stop-stage 5 --ngpu 1 --lang fa --local_data_opts --lang fa --use_lm true --lm_config conf/train_lm.yaml --token_type char --nbpe 150 --feats_type raw --speed_perturb_factors 0.9 1.0 1.1 --asr_config conf/train_lm.yaml --inference_config conf/train_lm.yaml --train_set train_fa --valid_set dev_fa --test_sets test_fa
[INFO] /content/espnet/egs2/commonvoice/asr1/../../../tools/activate_python.sh is not present
2023-08-13T12:45:02 (asr.sh:559:main) Skipped stages:  9 14 15 16 
2023-08-13T12:45:02 (asr.sh:915:main) Stage 5: Generate character level token_list from dump/raw/org/train_fa/text
/usr/bin/python3 /content/espnet/espnet2/bin/tokenize_text.py --token_type char --input dump/raw/lm_train.txt --output data/fa_token_list/char/tokens.txt --non_linguistic_symbols none --field 2- --cleaner none --g2p none --write_vocabulary true --add_symbol '<blank>:0' --add_symbol '<unk>:1' --add_symbol '<sos/eos>:-1'
2023-08-13 12:50:48,374 (to

In [ ]:
!./asr.sh --stage 6 --stop-stage 6 --ngpu 1 --lang "fa" --local_data_opts "--lang fa" --use_lm true --lm_config "conf/train_lm.yaml"  --token_type char --feats_type raw  --speed_perturb_factors "0.9 1.0 1.1"  --asr_config "conf/train_lm.yaml"  --inference_config "conf/train_lm.yaml" --train_set "train_fa"  --valid_set "dev_fa"  --test_sets "test_fa"  --bpe_train_text "/content/data/parsa.txt"  --lm_train_text "/content/data/parsa.txt"

2023-08-13T12:52:53 (asr.sh:277:main) ./asr.sh --stage 6 --stop-stage 6 --ngpu 1 --lang fa --local_data_opts --lang fa --use_lm true --lm_config conf/train_lm.yaml --token_type char --feats_type raw --speed_perturb_factors 0.9 1.0 1.1 --asr_config conf/train_lm.yaml --inference_config conf/train_lm.yaml --train_set train_fa --valid_set dev_fa --test_sets test_fa --bpe_train_text /content/data/parsa.txt --lm_train_text /content/data/parsa.txt
[INFO] /content/espnet/egs2/commonvoice/asr1/../../../tools/activate_python.sh is not present
2023-08-13T12:52:53 (asr.sh:559:main) Skipped stages:  9 14 15 16 
2023-08-13T12:52:53 (asr.sh:1006:main) Stage 6: LM collect stats: train_set=dump/raw/lm_train.txt, dev_set=dump/raw/org/dev_fa/text
2023-08-13T12:53:23 (asr.sh:1038:main) Generate 'exp/lm_stats_fa_char/run.sh'. You can resume the process from stage 6 using this script
2023-08-13T12:53:23 (asr.sh:1042:main) LM collect-stats started... log: 'exp/lm_stats_fa_char/logdir/stats.*.log'
run.pl: 32

In [ ]:
!./asr.sh --stage 7 --stop-stage 7 --ngpu 1 --lang "fa" --local_data_opts "--lang fa" --use_lm true --lm_config "conf/train_lm.yaml"  --token_type char --feats_type raw  --speed_perturb_factors "0.9 1.0 1.1"  --asr_config "conf/train_lm.yaml"  --inference_config "conf/train_lm.yaml" --train_set "train_fa"  --valid_set "dev_fa"  --test_sets "test_fa"  --bpe_train_text "/content/data/parsa.txt"  --lm_train_text "/content/data/parsa.txt"

2023-08-13T12:57:30 (asr.sh:277:main) ./asr.sh --stage 7 --stop-stage 7 --ngpu 1 --lang fa --local_data_opts --lang fa --use_lm true --lm_config conf/train_lm.yaml --token_type char --feats_type raw --speed_perturb_factors 0.9 1.0 1.1 --asr_config conf/train_lm.yaml --inference_config conf/train_lm.yaml --train_set train_fa --valid_set dev_fa --test_sets test_fa --bpe_train_text /content/data/parsa.txt --lm_train_text /content/data/parsa.txt
[INFO] /content/espnet/egs2/commonvoice/asr1/../../../tools/activate_python.sh is not present
2023-08-13T12:57:30 (asr.sh:559:main) Skipped stages:  9 14 15 16 
2023-08-13T12:57:30 (asr.sh:1083:main) Stage 7: LM Training: train_set=dump/raw/lm_train.txt, dev_set=dump/raw/org/dev_fa/text
2023-08-13T12:57:30 (asr.sh:1120:main) Generate 'exp/lm_train_lm_fa_char/run.sh'. You can resume the process from stage 7 using this script
2023-08-13T12:57:30 (asr.sh:1123:main) LM training started... log: 'exp/lm_train_lm_fa_char/train.log'
2023-08-13 12:57:30,277

# Train

In [ ]:
# 15 seconds
!./asr.sh --stage 10 --stop_stage 10 --nj 4 --train_set train_fa --valid_set dev_fa --test_sets test_fa --lang "fa" --local_data_opts "--lang fa" --asr_config conf/tuning/train_asr_transformer.yaml --nbpe 80

2023-07-30T08:47:42 (asr.sh:277:main) ./asr.sh --stage 10 --stop_stage 10 --nj 4 --train_set train_fa --valid_set dev_fa --test_sets test_fa --lang fa --local_data_opts --lang fa --asr_config conf/tuning/train_asr_transformer.yaml --nbpe 80
[INFO] /content/espnet/egs2/commonvoice/asr1/../../../tools/activate_python.sh is not present
2023-07-30T08:47:42 (asr.sh:559:main) Skipped stages:  9 14 15 16 
2023-07-30T08:47:42 (asr.sh:1182:main) Stage 10: ASR collect stats: train_set=dump/raw/train_fa, valid_set=dump/raw/dev_fa
2023-07-30T08:47:42 (asr.sh:1233:main) Generate 'exp/asr_stats_raw_fa_bpe80/run.sh'. You can resume the process from stage 10 using this script
2023-07-30T08:47:42 (asr.sh:1237:main) ASR collect-stats started... log: 'exp/asr_stats_raw_fa_bpe80/logdir/stats.*.log'


In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Launch tensorboard before training
%tensorboard --logdir /content/espnet/egs2/commonvoice/asr1/exp

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# It takes 12 minutes with a single T4 GPU.
!./asr.sh --stage 11 --stop_stage 11 --train_set train_fa --valid_set dev_fa --test_sets test_fa --lang "fa" --local_data_opts "--lang fa" --ngpu 1 --nj 4 --asr_config conf/tuning/train_asr_branchformer.yaml --nbpe 58

2023-07-30T09:14:19 (asr.sh:277:main) ./asr.sh --stage 11 --stop_stage 11 --train_set train_fa --valid_set dev_fa --test_sets test_fa --lang fa --local_data_opts --lang fa --ngpu 1 --nj 4 --asr_config conf/tuning/train_asr_transformer.yaml --nbpe 80
[INFO] /content/espnet/egs2/commonvoice/asr1/../../../tools/activate_python.sh is not present
2023-07-30T09:14:19 (asr.sh:559:main) Skipped stages:  9 14 15 16 
2023-07-30T09:14:19 (asr.sh:1295:main) Stage 11: ASR Training: train_set=dump/raw/train_fa, valid_set=dump/raw/dev_fa
2023-07-30T09:14:19 (asr.sh:1387:main) Generate 'exp/asr_train_asr_transformer_raw_fa_bpe80/run.sh'. You can resume the process from stage 11 using this script
2023-07-30T09:14:19 (asr.sh:1391:main) ASR training started... log: 'exp/asr_train_asr_transformer_raw_fa_bpe80/train.log'
2023-07-30 09:14:20,017 (launch:94) INFO: /usr/bin/python3 /content/espnet/espnet2/bin/launch.py --cmd 'run.pl --name exp/asr_train_asr_transformer_raw_fa_bpe80/train.log' --log exp/asr_tr

# Decode

In [ ]:
!./asr.sh --use_lm false --gpu_inference true --inference_nj 1 --stage 12 --stop_stage 12 --train_set train_fa --valid_set dev_fa --test_sets test_fa --asr_exp exp/"Change needede" --inference_config conf/tuning/decode_transformer.yaml --nbpe 58 --ngpu 1

# Evaluation